# df_points with UNIX Timestamp #
## Creates df_points.csv with all points and idletimes #
## Checks the valid geo location
## deletes the invalid points: 
2019 - ca. 9.5%

2020 - ca. 2.7%

### ca 1h for 1 year

In [17]:
def importdata():

    # show all rows
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # import Data
    df01 = pd.read_csv('/Users/jonweske/devment/BA_nextbike/data/lendings_2018.csv')


    df01 = df01[['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

    # cut
    #df01 = df01.truncate(after=10000)
    return df01

In [18]:
def dicOutOfDf(df01):
    import pandas as pd
    from tqdm import tqdm
    # Dict out of DataFrames of Trips per Bike

    # init DFs for every bike
    myDFs = {}

    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] not in myDFs:
            myDFs[df01['bike_id'][ind]] = pd.DataFrame(
                columns=['bike_id', 'start_time', 'end_time', 'start_lat', 'start_lng', 'end_lat', 'end_lng'])

    return myDFs

In [19]:
def addRentalsToDf(df01,myDFs):
    from tqdm import tqdm
    # Add all rentals to their bike_id DF
    for ind in tqdm(df01.index):
        if df01['bike_id'][ind] in myDFs:
            myDFs[df01['bike_id'][ind]].loc[df01.index[ind]] = df01.iloc[ind]
    return myDFs

In [20]:
def sortDFs(myDFs):
    # sort every DF
    for df in myDFs.values():
        df[['bike_id', 'start_time','end_time']] = df[['bike_id', 'start_time','end_time']].astype(int)
        df[['start_lat','start_lng', 'end_lat', 'end_lng']] = df[['start_lat','start_lng', 'end_lat', 'end_lng']].astype(float)
        df.sort_values(by=['start_time'], inplace=True)
        df.reset_index(drop=True, inplace=True)
    return myDFs

In [21]:
def dfPoints(myDFs,radius):
    # DF of all points with stoodtime
    inval = 0
    val = 0 
    
    df_points = pd.DataFrame(columns=['bike_id', 'lat', 'lng', 'timestood', 'time_start', 'time_end'])
    df_points[['bike_id', 'timestood', 'time_start', 'time_end']] = df_points[['bike_id', 'timestood', 'time_start', 'time_end']].astype(int)
    df_points[['lng', 'lat']] = df_points[['lng', 'lat']].astype(float)
    
    for df in tqdm(myDFs.values()):
        for ind in df.index[1:]:
            #in SECOUNDS
            timestood = (df['start_time'][ind] - df['end_time'][ind - 1]).astype(int)
            distance = qick_distance(df['end_lat'][ind-1], df['end_lng'][ind-1], df['start_lat'][ind], df['start_lng'][ind])
            if distance > radius:
                inval +=1
                continue
            val +=1
            dict = {'bike_id': df['bike_id'][ind], 'lng': df['start_lng'][ind], 'lat': df['start_lat'][ind],
                    'timestood': timestood, 'time_start': df['end_time'][ind - 1], 'time_end': df['start_time'][ind]}
            df_points = df_points.append(dict, ignore_index=True)
    print(f"invalides: {inval} from {val} thats {inval/val * 100} %")
    return df_points

In [22]:
#old
def diff(Elat, Elng, Slat, Slng):
    xdiff = abs(round(Elat,8)-round(Slat,8)) * 100000
    ydiff = abs(round(Elng,8)-round(Slng,8)) * 100000
    #print(f"{xdiff} und {ydiff}")
    return xdiff,ydiff

In [23]:
# returns distance in meter
# source: https://www.it-swarm.com.de/de/python/wie-kann-ich-die-entfernung-zwischen-zwei-punkten-breitengrad-laengengrad-schnell-schaetzen/1072488907/
def qick_distance(Lat1, Long1, Lat2, Long2):
    x = Lat2 - Lat1
    y = (Long2 - Long1)*cos((Lat2 + Lat1)*0.00872664626)  
    return 111.138*sqrt(x*x+y*y)*1000

In [24]:
def keplerMap(df_points):
    heatmap = KeplerGl()
    heatmap.add_data(data=df_points, name='points')
    heatmap.save_to_html(file_name='heatmap_test.html')

# Main 
## Create MyDfs Map

In [25]:
from keplergl import KeplerGl
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from geopy.distance import geodesic
import h3

df = pd.DataFrame()

df = importdata()
myDfs = dicOutOfDf(df)
myDfs = addRentalsToDf(df, myDfs)
myDfs = sortDFs(myDfs)


100%|██████████████████████████████████| 182199/182199 [03:52<00:00, 785.15it/s]


## Save MyDfs Map to file 

import pickle
with open('saved_myDfs_2019.pkl', 'wb') as f:
    pickle.dump(myDfs, f)

## Create DF_points 
### without ~9.573 % invalid points
52523 from 550865

def importMyDFsDic():
    with open('../data/saved_myDfs.pkl', 'rb') as f:
        myDfs = pickle.load(f)
    return myDfs

# ggf Map from file importieren
import pickle
myDf_imported = importMyDFsDic()

In [26]:
# create DF_POints
from math import radians, cos, sin, asin, sqrt
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm import tqdm

radius = 20
df_points = dfPoints(myDfs,radius)

100%|███████████████████████████████████████| 1300/1300 [04:34<00:00,  4.73it/s]

invalides: 41733 from 139166 thats 29.98792808588305 %


## Save df_points_valid_geov01.csv

In [27]:
df_points.to_csv("../data/df_points_2019_v1.csv",index=False)

df_points.head()

# Split Points at midnight

In [28]:
df_points['timestood'] = pd.to_timedelta(df_points['timestood'],unit='s')
df_points['time_start'] = pd.to_datetime(df_points['time_start'],unit='s')
df_points['time_end'] = pd.to_datetime(df_points['time_end'],unit='s')

In [29]:
from tqdm import tqdm
from datetime import timedelta, datetime, date

changed = 1
iterations = 0

while changed:
    iterations += 1
    changed = 0
    for ind in tqdm(df_points.index):
        ts = df_points['time_start'][ind]
        te = df_points['time_end'][ind]
        t0 = ts.replace(hour=23, minute=59, second=59)
        
        # is timestood greater than same date 23:59:59
        #   -> over midnight
        if ts + df_points['timestood'][ind] > t0:
            changed = 1
            
            underhang = t0 - ts
            overhang = te - t0
            
            # fix current day
            df_points['time_end'][ind] = ts.replace(hour=23, minute=59, second=59)
            df_points['timestood'][ind] = underhang
            
            # add new row
            time_start = ts.replace(hour=0, minute=0, second=0)
            time_start += timedelta(days=1)
            
            flag = 0
            if time_start + overhang > time_start.replace(hour=23, minute=59, second=59):
                flag = 1
            
            dict = {'bike_id': df_points['bike_id'][ind], 'lng': df_points['lng'][ind], 'lat': df_points['lat'][ind],
                    'timestood': overhang, 'time_start': time_start, 'time_end': te, 'flag' : flag}
            df_points = df_points.append(dict, ignore_index=True)
print(iterations)

100%|████████████████████████████████| 215476/215476 [00:09<00:00, 23654.04it/s]


100%|████████████████████████████████| 215493/215493 [00:08<00:00, 25953.40it/s]

118


In [30]:
for ind in tqdm(df_points.index):
    df_points['timestood'][ind] = df_points['timestood'][ind].seconds / 60

100%|██████████████████████████████████| 215493/215493 [04:38<00:00, 774.80it/s]


In [31]:
df_points[['timestood']] = df_points[['timestood']].astype(int)

# Save V2 df_points split

In [32]:
df_points.to_csv("../data/df_points_2019_v2.csv",index=False)

In [33]:
df_points['timestood'].max()

1439

## Add H3 Index

In [35]:
import h3

#dfh3 = pd.read_csv('../data/df_points_2020_v2.csv')
#dfh3['hex_id'] = 0
#dfh3.head()

#df_points['hex_id'] = 0

for ind in tqdm(df_points.index):
    df_points['hex_id'][ind] = h3.geo_to_h3(df_points['lat'][ind], df_points['lng'][ind], 8)
df_points.head()

100%|████████████████████████████████| 215493/215493 [00:03<00:00, 55375.55it/s]


,bike_id,lat,lng,timestood,time_start,time_end,flag,hex_id
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,NaN,881f1a8d91fffff
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,NaN,881f1a8cb1fffff
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,NaN,881f1a8cbdfffff
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,NaN,881f1a164bfffff
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,NaN,881f1a8da3fffff


In [38]:
#del df_points['flag']
df_points.head()

,bike_id,lat,lng,timestood,time_start,time_end,hex_id
0,3694.0,51.323619,12.335887,12,2017-12-31 23:47:09,2017-12-31 23:59:59,881f1a8d91fffff
1,3694.0,51.343813,12.381495,608,2018-01-01 13:51:06,2018-01-01 23:59:59,881f1a8cb1fffff
2,3694.0,51.343076,12.365177,35,2018-01-02 14:55:53,2018-01-02 15:31:44,881f1a8cbdfffff
3,3694.0,51.332874,12.373261,66,2018-01-02 22:53:45,2018-01-02 23:59:59,881f1a164bfffff
4,3694.0,51.297564,12.293015,424,2018-01-03 16:55:03,2018-01-03 23:59:59,881f1a8da3fffff


# Save V3 df_points H3

In [39]:
df_points.to_csv('../data/df_points_2019_final.csv', index=False)